# QE input script generator
This will automate the process of writing input scripts for Quantum Espresso to do the structural optimization of functionalized mofs using the pw.x (plane wave) package and pseudopotential files. 

**NOTE:** many default arguements will be preset.

In [1]:
using PorousMaterials
using Printf

In [2]:
@eval PorousMaterials PATH_TO_DATA = joinpath(pwd(), "..", "data")
@eval PorousMaterials PATH_TO_CRYSTALS = joinpath(pwd(), "..", "mof_construction", "NiPyC2_relax_sc211")

"/home/ng/DTRA/structural_relaxation/../mof_construction/NiPyC2_relax_sc211"

In [3]:
pwd()

"/home/ng/DTRA/structural_relaxation"

In [4]:
atomic_masses = read_atomic_masses();

In [5]:
# relabel Ni atoms
# @warn "we labeled Ni's 1-4, but not sure if we need to relabel as Ni1, Ni2 and have two of each."
# ni_count = 0
# for (i, atom) in enumerate(crystal.atoms.species)
#     if atom == :Ni
#         ni_count += 1
#         cnt = mod(ni_count, 2) + 1
#         crystal.atoms.species[i] = Symbol(@sprintf("Ni%d", cnt))
#     end
# end

## paths to pseudo-potential libraries

populate dictionary with names of pseudopotential files

In [6]:
functional_to_atom_to_pp_file = Dict{String, Dict{Symbol, String}}()

# dictionary for pbesol exchange correlation: pbesol
functional_to_atom_to_pp_file["pbesol"] = Dict{Symbol, String}()
 
functional_to_atom_to_pp_file["pbesol"][:Ni] = "Ni.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:O]  = "O.pbesol-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:C]  = "C.pbesol-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:N]  = "N.pbesol-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:H]  = "H.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:F]  = "F.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:Cl] = "Cl.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:Br] = "Br.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:P]  = "P.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:S]  = "S.pbesol-n-nc.UPF"

# dictionary for VDW exchange correlation: vdw-df or vdw-df2?
functional_to_atom_to_pp_file["vdw-df2"] = Dict{Symbol, String}()

functional_to_atom_to_pp_file["vdw-df2"][:Ni] = "Ni.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["vdw-df2"][:O]  = "O.pbesol-nc.UPF"
functional_to_atom_to_pp_file["vdw-df2"][:C]  = "C.pbesol-nc.UPF"
functional_to_atom_to_pp_file["vdw-df2"][:N]  = "N.pbesol-nc.UPF"
functional_to_atom_to_pp_file["vdw-df2"][:H]  = "H.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["vdw-df2"][:F]  = "F.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["vdw-df2"][:Cl] = "Cl.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["vdw-df2"][:Br] = "Br.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["vdw-df2"][:P]  = "P.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["vdw-df2"][:S]  = "S.pbesol-n-nc.UPF"

"S.pbesol-n-nc.UPF"

## default QE structural relaxation params

In [7]:
"""
    input_params = qe_input_params(crystal; kwargs)

create a dictionary consisting of input arguments for QE structural relaxation
needed for the script and set default values
then populate with values specific to each material
link with full list and description of input params
https://www.quantum-espresso.org/Doc/INPUT_PW.html#__top__

WARNING: tailored to Nickel
"""
function qe_structural_opt_input_params(crystal::Crystal; kwargs...)
    @warn "tailored to NiPyC2 and their analogues"
    @assert :Ni in crystal.atoms.species
    
    input_params = Dict{Symbol, Any}()
    
    # &control params
    input_params[:calculation] = "relax" # vc-relax
    input_params[:functional] = "pbesol" # vdw-df2
    
    input_params[:prefix] = split(crystal.name, ".")[1] * "_" *
    input_params[:functional] * "_" * input_params[:calculation]
    
    input_params[:pseudo_dir] = "./pseudo/" 
#     input_params[:outdir] = joinpath("./QE_relaxation_results/",
#         input_params[:functional] * "_" * input_params[:calculation])
    input_params[:outdir] = "/nfs/hpc/share/gantzlen"
    input_params[:wf_collect] = ".FALSE."
    input_params[:nstep] = 200

    # &SYSTEM params
    species = [Symbol(split(String(s), "fg")[1]) for s in crystal.atoms.species]
    input_params[:ibrav] = 0
    input_params[:nat] = crystal.atoms.n
    input_params[:ntyp] = length(unique(species))
    input_params[:tot_charge] = 0.0
    input_params[:ecutwfc] = 100.0
        # input_dft default: read from pseudo file
    input_params[:input_dft] = "" # "! input_dft = 'vdw-df2'" 
    input_params[:vdw_corr] = "grimme-d2"
    input_params[:occupations] = "smearing"
    input_params[:degauss] = 0.2
    input_params[:smearing] = "mv"
    input_params[:nspin] = 2
    input_params[:magnetic_species] = :Ni
    input_params[:unpaired_e_per_mag_species] = 2
    input_params[:sub_lat_species_index] = findall(sort(unique(species)) .== 
                                            input_params[:magnetic_species])[1]
    input_params[:starting_magnetization] = 0.25
    # tot_mag = total number of unpaired electrons in sys 
    input_params[:tot_magnetization] = sum(species .== input_params[:magnetic_species]) * 
                                       input_params[:unpaired_e_per_mag_species]

    # &ELECTRON params
    input_params[:electron_maxstep] = 90
    input_params[:scf_must_converge] = ".TRUE."
    input_params[:conv_thr] = "1.D-6"
    input_params[:mixing_mode] = "local-TF"
    input_params[:diagonalization] = "david"

    # &IONS params
    input_params[:ion_dynamics] = "bfgs"

    # &CELL params
    input_params[:cell_dynamics] = "bfgs"
    input_params[:cell_dofree] = "xyz"

    # K_POINTS params
    input_params[:k_points] = [String("{gamma}")]
    
    # over-ride default params with ones passed in kwargs
    for (key, value) in kwargs
        input_params[key] = value
    end
    return input_params
end
# input_params = qe_structural_opt_input_params(crystal)

qe_structural_opt_input_params

In [8]:
function check_psuedopotential_coverage(crystal::Crystal, functional::String)
    species = [Symbol(split(String(s), "fg")[1]) for s in crystal.atoms.species]
    for atom in unique(species)
        if ! (atom in keys(functional_to_atom_to_pp_file[functional]))
            error(@sprintf("atom %s not in `atom_to_psuedopotential_file`", atom))
        end
    end
end

# check_psuedopotential_coverage(crystal, "pbesol") # vdw-df2

check_psuedopotential_coverage (generic function with 1 method)

In [9]:
function generate_qe_structural_opt_input_file(crystal::Crystal, 
                                               input_params::Dict{Symbol, Any}; 
                                               freeze_parent_atoms::Bool=false,
                                               second_stage::Bool=false)
    # check pseudo-potential coverage
    check_psuedopotential_coverage(crystal, input_params[:functional])
    
    # determine input file name
    if input_params[:cell_dofree] == "all"
        qe_input_filename = "pw." * split(crystal.name, ".cif")[1] * 
        "_" * input_params[:functional] * "_angle-" * input_params[:calculation] * ".in"
    else
        qe_input_filename = "pw." * split(crystal.name, ".cif")[1] *
        "_" * input_params[:functional] * "_" * input_params[:calculation] * ".in"
    end
    
    # modifiy filename to indicate special casses
    if freeze_parent_atoms     
         # parent atoms are frozen in structure
        qe_input_filename = split(qe_input_filename, ".in")[1] * "_frozen_parent_atoms.in"
    elseif second_stage
        # on second round of relaxation
        qe_input_filename = "pw." * split(crystal.name, ".cif")[1] * "_Round2.in"
    end
    
    println(qe_input_filename)
    
    # make this folder if it doesn't exist
    if ! isdir("QE_input_scripts")
        mkdir("QE_input_scripts")
    end
    qe_input_file = open(joinpath("QE_input_scripts", qe_input_filename)
        , "w")

    # write &Control
    @printf(qe_input_file,
        """
        &CONTROL
           calculation = '%s',
           prefix = '%s',
           pseudo_dir = '%s',
           outdir = '%s'
           wf_collect = %s,
           nstep = %.0f,
        /
        """,
        input_params[:calculation],
        input_params[:prefix],
        input_params[:pseudo_dir],
        input_params[:outdir],
        input_params[:wf_collect],
        input_params[:nstep]
    )

    # write &SYSTEM
    @printf(qe_input_file,
        """
        &SYSTEM
           ibrav = %.0f,
           nat = %.0f,
           ntyp = %0.f,
           tot_charge = %f,
           ecutwfc = %f,
           %s
           vdw_corr = '%s',
           occupations = '%s',
           degauss = %f,
           smearing = '%s',
           nspin = %.0f ,
           starting_magnetization(%d) = %f,
           tot_magnetization = %f
        /
        """, 
        input_params[:ibrav], input_params[:nat],
        input_params[:ntyp], input_params[:tot_charge],
        input_params[:ecutwfc],
        input_params[:input_dft], 
        input_params[:vdw_corr],
        input_params[:occupations], input_params[:degauss],
        input_params[:smearing], input_params[:nspin],
        input_params[:sub_lat_species_index],
        input_params[:starting_magnetization][1],
        input_params[:tot_magnetization]
    )

    # write &ELECTRONS
    @printf(qe_input_file,
        """
        &ELECTRONS
            electron_maxstep = %0.f,
            scf_must_converge = %s,
            conv_thr = %s,
            mixing_mode = '%s',
            diagonalization = '%s',
        /
        """,
        input_params[:electron_maxstep],
        input_params[:scf_must_converge],
        input_params[:conv_thr],
        input_params[:mixing_mode], 
        input_params[:diagonalization]
    )

    # write &IONS
    @printf(qe_input_file,
        """
        &IONS
           ion_dynamics = '%s',
        /
        """,
        input_params[:ion_dynamics]
    )


    # write &CELL
    @printf(qe_input_file,
        """
        &CELL
           cell_dynamics = '%s'
           cell_dofree = '%s'
        /
        """,
        input_params[:cell_dynamics],
        input_params[:cell_dofree]
    )

    # write ATOMIC_SPECIES
    @printf(qe_input_file,
        """
        ATOMIC_SPECIES
        """
    )
    
    species = [Symbol(split(String(s), "fg")[1]) for s in crystal.atoms.species]
    for atom in sort(unique(species))
        @printf(qe_input_file,
            """
            %s    %f    %s
            """,
            atom, atomic_masses[atom], 
            functional_to_atom_to_pp_file[input_params[:functional]][atom]
        )
    end

    # write K_POINTS
    @printf(qe_input_file,
        """
        K_POINTS %s
        """,
        input_params[:k_points][1]
    )


    # write CELL_PARAMETERS
    @printf(qe_input_file,
        """
        CELL_PARAMETERS {angstrom}
        """
    )
    for d = 1:3
        @printf(qe_input_file,
            "     %f   %f   %f\n",
            crystal.box.f_to_c[1, d], 
            crystal.box.f_to_c[2, d],
            crystal.box.f_to_c[3, d]
        )
    end

    # write ATOMIC_POSITIONS
    #symbol #x #y #z
    @printf(qe_input_file, "ATOMIC_POSITIONS {crystal}\n")

    id_sorted = sortperm(crystal.atoms.species)
    for a in id_sorted
        species = String(crystal.atoms.species[a])
        # write atomic positions for different cases
        # Case 1: freeze the atoms of the parent MOF
        #         and only relax the atoms of the fragment
        #
        # Case 2: allow all atoms in the MOF the be relaxed
        if freeze_parent_atoms
            if occursin("fg", species)
                species = split(species, "fg")[1]
                @printf(qe_input_file, 
                   "%s %f  %f  %f 1 1 1\n", 
                   species,
                   crystal.atoms.coords.xf[1, a], 
                   crystal.atoms.coords.xf[2, a], 
                   crystal.atoms.coords.xf[3, a]
                )
            else
                @printf(qe_input_file, 
                   "%s %f  %f  %f 0 0 0\n", 
                   species,
                   crystal.atoms.coords.xf[1, a], 
                   crystal.atoms.coords.xf[2, a], 
                   crystal.atoms.coords.xf[3, a]
                )
            end
        else
            @printf(qe_input_file, 
                   "%s %f  %f  %f\n", 
                   species,
                   crystal.atoms.coords.xf[1, a], 
                   crystal.atoms.coords.xf[2, a], 
                   crystal.atoms.coords.xf[3, a]
                )
        end
    end
    close(qe_input_file)
    return qe_input_filename
end

generate_qe_structural_opt_input_file (generic function with 1 method)

# Generate Input File

## TODO:
- make vdw-df2 input files for simple structures
- figure out how to do 'vc-relax' using vdw-df2
- make input files with frozen parent mof:
    - use calculation='relax' on first run
    - set scf_must_converge=.false.
    - set conv_thr = 1d-5 or 5d-5
    - set electron_maxstep = 40
- make 2nd round relaxation code cleaner

In [10]:
# The name of the fragment
# file must be a .xyz file located in ./fragments/
# The atom species that is bonded to the
# :C_aro_R atom on the ring segment of the Fragment
fragment_list = [
    "Br",
#     "C-3CH3",
    "C-C",
#     "CH2-CH2-CH3",
#     "CH2-CH3",
#     "CH2-NH2",
    "CH3",
#     "CH-CH2",
    "CH-O",
#     "CH-S",
    "Cl",
    "C-N",
    "F",
#     "N-2CH3",
    "N-C-O",
    "NH2",
    "N-NH",
#     "O-CH2-CH2-CH3",
#     "O-CH2-CH3",
#     "O-CH3",
    "O-C-N",
    "OH",
    "O-OH",
#     "P-2CH3",
#     "PH2",
#     "S-CH3",
#     "SH"
]


fragments_with_overlap = ["C-3CH3", "CH2-CH2-CH3", "CH2-CH3","CH2-NH2",
                          "CH-CH2", "CH-S", "N-2CH3", "O-CH2-CH2-CH3",
                          "O-CH3", "O-CH2-CH3", "P-2CH3", "PH2", 
                          "S-CH3", "SH"]

# The type of Arene Substitution ("ortho", "meta", "all") 
# TODO: "all" not yet implimented
# substitution_types = ["meta", "ortho"] 
substitution_types = ["meta"] 

1-element Array{String,1}:
 "meta"

In [16]:
functional = "vdw-df2" # "pbesol" or "vdw-df2"(VdW only works for relax)
calculation = "relax"
#####
# flag to determine whether xtals are going though a second stage of relaxation
#####
second_round = false


for sub_type in substitution_types
    for fragment_name in fragment_list
        ####
        # set flag for overlaping atoms
        ####
        has_overlap = false
        if (fragment_name in fragments_with_overlap && !second_round)
            # determine name of crystal using rules from MOFun.jl
            crystal_name = "NiPyC2_relax_sc211_" * sub_type * "_functionalized_" * fragment_name * ".cif"

            # read in the crystal
            crystal = Crystal(crystal_name)
            strip_numbers_from_atom_labels!(crystal)
            
            # pass flag for freezing parent atoms
            has_overlap=true 
            # indicate special case in filename
            file_prefixes = split(crystal.name, ".")[1] * "_" * functional * "_" * 
                            calculation * "_frozen_parent_atoms"

            input_params =  qe_structural_opt_input_params(crystal, 
                                                           prefix=file_prefixes,
                                                           electron_maxstep=40, 
                                                           scf_must_converge=".FALSE.",
                                                           conv_thr="5.D-6") 
        elseif (fragment_name in fragments_with_overlap && second_round)
            ####
            # Special Treatment for xtals going through 2nd round of relaxation ###
            ####
            crystal_name = "NiPyC2_relax_sc211_" * sub_type * "_functionalized_" *
                            fragment_name * "_" * functional * "_" * 
                            calculation * "_frozen_parent_atoms.cif" 
            
            # read in the crystal
            @eval PorousMaterials PATH_TO_CRYSTALS = joinpath(pwd(), "post-relaxation_cifs")
            crystal = Crystal(crystal_name)
            strip_numbers_from_atom_labels!(crystal)
            # indicate special case in filename
            file_prefixes = split(crystal.name, ".")[1] * "_Round2"
            # second round relaxation has the same params as "ordinary" relaxations
            input_params =  qe_structural_opt_input_params(crystal, prefix=file_prefixes)
            
        else
            # determine name of crystal using rules from MOFun.jl
            crystal_name = "NiPyC2_relax_sc211_" * sub_type * "_functionalized_" * fragment_name * ".cif"

            # read in the crystal
            crystal = Crystal(crystal_name)
            strip_numbers_from_atom_labels!(crystal)
            input_params = qe_structural_opt_input_params(crystal)
        end
        if functional == "vdw-df2"
            file_prefixe = split(crystal.name, ".")[1] * "_" * functional * "_" * calculation
            
            input_params = qe_structural_opt_input_params(crystal;
                        calculation=calculation,
                        prefix=file_prefixe,
                        functional=functional,
                        input_dft=(functional == "vdw-df2") ? "input_dft = 'vdw-df2'," : "")
        end
        
        ####
        # generate QE input file
        ####
        qe_input_filename =  generate_qe_structural_opt_input_file(crystal, input_params, 
                                                                   freeze_parent_atoms=has_overlap,
                                                                   second_stage=second_round)
    end
end

┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13


pw.NiPyC2_relax_sc211_meta_functionalized_Br_vdw-df2_relax.in
pw.NiPyC2_relax_sc211_meta_functionalized_C-C_vdw-df2_relax.in
pw.NiPyC2_relax_sc211_meta_functionalized_CH3_vdw-df2_relax.in
pw.NiPyC2_relax_sc211_meta_functionalized_CH-O_vdw-df2_relax.in
pw.NiPyC2_relax_sc211_meta_functionalized_Cl_vdw-df2_relax.in
pw.NiPyC2_relax_sc211_meta_functionalized_C-N_vdw-df2_relax.in
pw.NiPyC2_relax_sc211_meta_functionalized_F_vdw-df2_relax.in
pw.NiPyC2_relax_sc211_meta_functionalized_N-C-O_vdw-df2_relax.in
pw.NiPyC2_relax_sc211_meta_functionalized_NH2_vdw-df2_relax.in
pw.NiPyC2_relax_sc211_meta_functionalized_N-NH_vdw-df2_relax.in


┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogue

pw.NiPyC2_relax_sc211_meta_functionalized_O-C-N_vdw-df2_relax.in
pw.NiPyC2_relax_sc211_meta_functionalized_OH_vdw-df2_relax.in
pw.NiPyC2_relax_sc211_meta_functionalized_O-OH_vdw-df2_relax.in


┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13
┌ Warning: tailored to NiPyC2 and their analogues
└ @ Main In[7]:13


In [12]:
# # input_params = qe_structural_opt_input_params(crystal)

# calculation_type = "relax"
# exchange_correlation = "vdw-df2"

# file_prefixes =  split(crystal.name, ".")[1] * "_" *
#     exchange_correlation * "_" * calculation_type
    
# output_directory = joinpath("./QE_relaxation_results/",
#         exchange_correlation * "_" * calculation_type)
    
# input_params = qe_structural_opt_input_params(crystal;
#                     calculation=calculation_type,
#                     prefix=file_prefixes,
#                     outdir=output_directory,
#                     functional=exchange_correlation,
#                     input_dft=(exchange_correlation == "vdw-df2") ? "input_dft = 'vdw-df2'," : "", 
#                     freeze_parent_atoms=true);

In [13]:
# run(`cat QE_input_scripts/$qe_input_filename`)